In [ ]:
#!pip install pyspark
#!pip install findspark

In [4]:
#!rm -rf ~/data/Predict-Forclosure*
#!mkdir -p ~/data
#!wget https://sadatashareagsparkml.blob.core.windows.net/data/Predict-Forclosure.zip -O ~/data/Predict-Forclosure.zip
#!unzip ~/data/Predict-Forclosure.zip -d ~/data/

--2022-06-01 23:53:42--  https://sadatashareagsparkml.blob.core.windows.net/data/Predict-Forclosure.zip
Resolving sadatashareagsparkml.blob.core.windows.net (sadatashareagsparkml.blob.core.windows.net)... 20.150.90.36
Connecting to sadatashareagsparkml.blob.core.windows.net (sadatashareagsparkml.blob.core.windows.net)|20.150.90.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52933103 (50M) [application/x-zip-compressed]
Saving to: ‘/home/atingupta2005/data/Predict-Forclosure.zip’

/home/atingupta2005 100%[===================>]  50.48M   161MB/s    in 0.3s    

2022-06-01 23:53:42 (161 MB/s) - ‘/home/atingupta2005/data/Predict-Forclosure.zip’ saved [52933103/52933103]

Archive:  /home/atingupta2005/data/Predict-Forclosure.zip
  inflating: /home/atingupta2005/data/Predict-Forclosure/Data_Definitionse3ea310.xlsx  
   creating: /home/atingupta2005/data/Predict-Forclosure/Foreclosure-Prediction-Dataset/
  inflating: /home/atingupta2005/data/Predict-Forclosure/Fo

In [6]:
!ls ~/data/Predict-Forclosure/Foreclosure-Prediction-Dataset

Customers_31JAN2019.csv  RF_Final_Data.csv     train_foreclosure.csv
LMS_31JAN2019.csv	 test_foreclosure.csv


In [21]:
from pathlib import Path
home = str(Path.home())

path_LMS = f"{home}/data/Predict-Forclosure/Foreclosure-Prediction-Dataset/LMS_31JAN2019.csv"
path_RMS_Final = f"{home}/data/Predict-Forclosure/Foreclosure-Prediction-Dataset/RF_Final_Data.csv"
path_Customer = f"{home}/data/Predict-Forclosure/Foreclosure-Prediction-Dataset/Customers_31JAN2019.csv"

In [22]:
import findspark
findspark.init("/usr/spark")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [23]:
df_lms = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv(path_LMS)
df_lms

DataFrame[AGREEMENTID: int, CUSTOMERID: string, LOAN_AMT: string, NET_DISBURSED_AMT: string, INTEREST_START_DATE: string, CURRENT_ROI: double, ORIGNAL_ROI: double, CURRENT_TENOR: int, ORIGNAL_TENOR: int, DUEDAY: int, AUTHORIZATIONDATE: string, CITY: string, PRE_EMI_DUEAMT: double, PRE_EMI_RECEIVED_AMT: double, PRE_EMI_OS_AMOUNT: double, EMI_DUEAMT: double, EMI_RECEIVED_AMT: double, EMI_OS_AMOUNT: double, EXCESS_AVAILABLE: double, EXCESS_ADJUSTED_AMT: double, BALANCE_EXCESS: double, NET_RECEIVABLE: double, OUTSTANDING_PRINCIPAL: double, PAID_PRINCIPAL: double, PAID_INTEREST: double, MONTHOPENING: double, LAST_RECEIPT_DATE: string, LAST_RECEIPT_AMOUNT: double, NET_LTV: double, COMPLETED_TENURE: int, BALANCE_TENURE: int, DPD: int, FOIR: double, PRODUCT: string, SCHEMEID: string, NPA_IN_LAST_MONTH: string, NPA_IN_CURRENT_MONTH: string, MOB: int]

In [24]:
df_RMS_Final = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv(path_RMS_Final)
df_RMS_Final

DataFrame[TicketId: string, Type: string, SubType: string, Status: string, Date: string, Preprocessed_EmailBody: string, Preprocessed_Subject: string, Masked_CustomerID: string, Masked_AgreementID: string]

In [25]:
df_Customer = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv(path_Customer)
df_Customer

DataFrame[CUSTOMERID: int, CUST_CONSTTYPE_ID: int, CUST_CATEGORYID: int, PROFESSION: string, AGE: int, SEX: string, MARITAL_STATUS: string, QUALIFICATION: string, NO_OF_DEPENDENT: int, OCCUPATION: string, POSITION: string, GROSS_INCOME: double, PRE_JOBYEARS: int, NETTAKEHOMEINCOME: double, BRANCH_PINCODE: string]

In [7]:
df_Customer.show(1)

+----------+-----------------+---------------+----------+---+---+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|CUSTOMERID|CUST_CONSTTYPE_ID|CUST_CATEGORYID|PROFESSION|AGE|SEX|MARITAL_STATUS|QUALIFICATION|NO_OF_DEPENDENT|OCCUPATION|POSITION|GROSS_INCOME|PRE_JOBYEARS|NETTAKEHOMEINCOME|BRANCH_PINCODE|
+----------+-----------------+---------------+----------+---+---+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|  12001000|                1|              5|      null| 33|  M|             M|     POSTGRAD|              0|      null|    null| 198375.2218|           8|      198375.2218|        400070|
+----------+-----------------+---------------+----------+---+---+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
only showing top 1 row



In [8]:
df_RMS_Final.show(1)

+--------+-------+--------------------+------+-------------------+----------------------+--------------------+-----------------+------------------+
|TicketId|   Type|             SubType|Status|               Date|Preprocessed_EmailBody|Preprocessed_Subject|Masked_CustomerID|Masked_AgreementID|
+--------+-------+--------------------+------+-------------------+----------------------+--------------------+-----------------+------------------+
|  571910|REQUEST|PROVISIONAL IT CE...| Close|2019-01-23 16:34:00|  hi team, can you ...|loan account numb...|         12020736|                 0|
+--------+-------+--------------------+------+-------------------+----------------------+--------------------+-----------------+------------------+
only showing top 1 row



In [9]:
df_lms.show(1)

+-----------+----------+----------------+-----------------+-------------------+-----------+-----------+-------------+-------------+------+-----------------+------+--------------+--------------------+-----------------+-----------+----------------+-------------+----------------+-------------------+--------------+--------------+---------------------+--------------+-------------+------------+-----------------+-------------------+-------+----------------+--------------+---+----+-------+--------+-----------------+--------------------+---+
|AGREEMENTID|CUSTOMERID|        LOAN_AMT|NET_DISBURSED_AMT|INTEREST_START_DATE|CURRENT_ROI|ORIGNAL_ROI|CURRENT_TENOR|ORIGNAL_TENOR|DUEDAY|AUTHORIZATIONDATE|  CITY|PRE_EMI_DUEAMT|PRE_EMI_RECEIVED_AMT|PRE_EMI_OS_AMOUNT| EMI_DUEAMT|EMI_RECEIVED_AMT|EMI_OS_AMOUNT|EXCESS_AVAILABLE|EXCESS_ADJUSTED_AMT|BALANCE_EXCESS|NET_RECEIVABLE|OUTSTANDING_PRINCIPAL|PAID_PRINCIPAL|PAID_INTEREST|MONTHOPENING|LAST_RECEIPT_DATE|LAST_RECEIPT_AMOUNT|NET_LTV|COMPLETED_TENURE|BALANCE

In [10]:
df_lms = df_Customer.repartition(6)

In [11]:
df_lms.show()

+----------+-----------------+---------------+----------+----+----+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|CUSTOMERID|CUST_CONSTTYPE_ID|CUST_CATEGORYID|PROFESSION| AGE| SEX|MARITAL_STATUS|QUALIFICATION|NO_OF_DEPENDENT|OCCUPATION|POSITION|GROSS_INCOME|PRE_JOBYEARS|NETTAKEHOMEINCOME|BRANCH_PINCODE|
+----------+-----------------+---------------+----------+----+----+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|  12012262|                1|              4|      null|  25|   M|             S|           UG|              0|      null|    null| 71186.43251|        null|      71186.43251|        500050|
|  12003305|                1|              4|      null|  45|   F|             M|         GRAD|              0|      null|    null| 0.120103664|        null|      0.124607551|        600017|
|  12008188|                1|          

In [12]:
df_lms.coalesce(4).show()

+----------+-----------------+---------------+----------+----+----+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|CUSTOMERID|CUST_CONSTTYPE_ID|CUST_CATEGORYID|PROFESSION| AGE| SEX|MARITAL_STATUS|QUALIFICATION|NO_OF_DEPENDENT|OCCUPATION|POSITION|GROSS_INCOME|PRE_JOBYEARS|NETTAKEHOMEINCOME|BRANCH_PINCODE|
+----------+-----------------+---------------+----------+----+----+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|  12012262|                1|              4|      null|  25|   M|             S|           UG|              0|      null|    null| 71186.43251|        null|      71186.43251|        500050|
|  12003305|                1|              4|      null|  45|   F|             M|         GRAD|              0|      null|    null| 0.120103664|        null|      0.124607551|        600017|
|  12008188|                1|          

In [13]:
df_lms = df_lms.repartition(1)
df_lms.rdd.getNumPartitions()

1

In [14]:
df_lms.sort("MARITAL_STATUS").explain()

== Physical Plan ==
*(2) Sort [MARITAL_STATUS#130 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(MARITAL_STATUS#130 ASC NULLS FIRST, 200)
   +- Exchange RoundRobinPartitioning(1)
      +- *(1) FileScan csv [CUSTOMERID#124,CUST_CONSTTYPE_ID#125,CUST_CATEGORYID#126,PROFESSION#127,AGE#128,SEX#129,MARITAL_STATUS#130,QUALIFICATION#131,NO_OF_DEPENDENT#132,OCCUPATION#133,POSITION#134,GROSS_INCOME#135,PRE_JOBYEARS#136,NETTAKEHOMEINCOME#137,BRANCH_PINCODE#138] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/atingupta2005/pyspark-mllib-barclays-june-22/Spark/03-Joining Spark ..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CUSTOMERID:int,CUST_CONSTTYPE_ID:int,CUST_CATEGORYID:int,PROFESSION:string,AGE:int,SEX:str...


In [15]:
df_lms.orderBy("MARITAL_STATUS").rdd.getNumPartitions()

4

In [16]:
df_lms_sort = df_lms.orderBy("NO_OF_DEPENDENT")

In [17]:
df_lms_sort.explain()

== Physical Plan ==
*(2) Sort [NO_OF_DEPENDENT#132 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(NO_OF_DEPENDENT#132 ASC NULLS FIRST, 200)
   +- Exchange RoundRobinPartitioning(1)
      +- *(1) FileScan csv [CUSTOMERID#124,CUST_CONSTTYPE_ID#125,CUST_CATEGORYID#126,PROFESSION#127,AGE#128,SEX#129,MARITAL_STATUS#130,QUALIFICATION#131,NO_OF_DEPENDENT#132,OCCUPATION#133,POSITION#134,GROSS_INCOME#135,PRE_JOBYEARS#136,NETTAKEHOMEINCOME#137,BRANCH_PINCODE#138] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/atingupta2005/pyspark-mllib-barclays-june-22/Spark/03-Joining Spark ..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CUSTOMERID:int,CUST_CONSTTYPE_ID:int,CUST_CATEGORYID:int,PROFESSION:string,AGE:int,SEX:str...


In [18]:
df_lms_sort.show()

+----------+-----------------+---------------+----------+----+----+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|CUSTOMERID|CUST_CONSTTYPE_ID|CUST_CATEGORYID|PROFESSION| AGE| SEX|MARITAL_STATUS|QUALIFICATION|NO_OF_DEPENDENT|OCCUPATION|POSITION|GROSS_INCOME|PRE_JOBYEARS|NETTAKEHOMEINCOME|BRANCH_PINCODE|
+----------+-----------------+---------------+----------+----+----+--------------+-------------+---------------+----------+--------+------------+------------+-----------------+--------------+
|  12001088|                2|              8|      null|null|null|          null|         null|           null|      null|    null|         0.0|        null|              0.0|          #N/A|
|  12003654|                1|              5|      null|null|null|          null|         null|           null|      null|    null|         0.0|        null|              0.0|          #N/A|
|  12001109|                6|          

In [19]:
print(df_lms.columns)
print(df_Customer.columns)

['CUSTOMERID', 'CUST_CONSTTYPE_ID', 'CUST_CATEGORYID', 'PROFESSION', 'AGE', 'SEX', 'MARITAL_STATUS', 'QUALIFICATION', 'NO_OF_DEPENDENT', 'OCCUPATION', 'POSITION', 'GROSS_INCOME', 'PRE_JOBYEARS', 'NETTAKEHOMEINCOME', 'BRANCH_PINCODE']
['CUSTOMERID', 'CUST_CONSTTYPE_ID', 'CUST_CATEGORYID', 'PROFESSION', 'AGE', 'SEX', 'MARITAL_STATUS', 'QUALIFICATION', 'NO_OF_DEPENDENT', 'OCCUPATION', 'POSITION', 'GROSS_INCOME', 'PRE_JOBYEARS', 'NETTAKEHOMEINCOME', 'BRANCH_PINCODE']


In [20]:
df_lms.alias("df_lms"). \
    join(df_Customer.alias("df_Customer"), df_lms.CUSTOMERID == df_Customer.CUSTOMERID). \
    select("df_lms.CUSTOMERID","df_Customer.SEX","df_Customer.MARITAL_STATUS","df_Customer.GROSS_INCOME","df_Customer.PRE_JOBYEARS").\
    show()

+----------+---+--------------+------------+------------+
|CUSTOMERID|SEX|MARITAL_STATUS|GROSS_INCOME|PRE_JOBYEARS|
+----------+---+--------------+------------+------------+
|  12001000|  M|             M| 198375.2218|           8|
|  12001001|  M|             M| 242703.9829|          10|
|  12001002|  M|             M|         0.0|        null|
|  12001003|  M|             M| 365263.5116|        null|
|  12001004|  M|             M|  187342.199|        null|
|  12001005|  M|             M| 53536.20823|        null|
|  12001006|  M|             M| 49842.14981|        null|
|  12001007|  M|             M| 123370.4837|           7|
|  12001008|  M|             M| 183331.9827|        null|
|  12001009|  M|             M| 93803.84407|        null|
|  12001010|  M|             M| 35880.96962|           5|
|  12001011|  M|             M| 63576.87454|           0|
|  12001012|  M|             M| 108093.2976|           1|
|  12001013|  M|             M| 331757.8472|           5|
|  12001014|  

In [21]:
# as both data frames have user id using same name, we can pass column name as string as well
df_lms.alias("df_lms"). \
    join(df_Customer.alias("df_Customer"), 'CUSTOMERID'). \
    select("df_lms.CUSTOMERID","df_Customer.SEX","df_Customer.MARITAL_STATUS","df_Customer.GROSS_INCOME","df_Customer.PRE_JOBYEARS").\
    show()

+----------+---+--------------+------------+------------+
|CUSTOMERID|SEX|MARITAL_STATUS|GROSS_INCOME|PRE_JOBYEARS|
+----------+---+--------------+------------+------------+
|  12001000|  M|             M| 198375.2218|           8|
|  12001001|  M|             M| 242703.9829|          10|
|  12001002|  M|             M|         0.0|        null|
|  12001003|  M|             M| 365263.5116|        null|
|  12001004|  M|             M|  187342.199|        null|
|  12001005|  M|             M| 53536.20823|        null|
|  12001006|  M|             M| 49842.14981|        null|
|  12001007|  M|             M| 123370.4837|           7|
|  12001008|  M|             M| 183331.9827|        null|
|  12001009|  M|             M| 93803.84407|        null|
|  12001010|  M|             M| 35880.96962|           5|
|  12001011|  M|             M| 63576.87454|           0|
|  12001012|  M|             M| 108093.2976|           1|
|  12001013|  M|             M| 331757.8472|           5|
|  12001014|  

In [22]:
# Using df_lms["CUSTOMERID"]
df_lms.alias("df_lms"). \
    join(df_Customer.alias("df_Customer"), 'CUSTOMERID'). \
    select(df_lms["CUSTOMERID"], "df_Customer.SEX","df_Customer.MARITAL_STATUS","df_Customer.GROSS_INCOME","df_Customer.PRE_JOBYEARS").\
    show()

+----------+---+--------------+------------+------------+
|CUSTOMERID|SEX|MARITAL_STATUS|GROSS_INCOME|PRE_JOBYEARS|
+----------+---+--------------+------------+------------+
|  12001000|  M|             M| 198375.2218|           8|
|  12001001|  M|             M| 242703.9829|          10|
|  12001002|  M|             M|         0.0|        null|
|  12001003|  M|             M| 365263.5116|        null|
|  12001004|  M|             M|  187342.199|        null|
|  12001005|  M|             M| 53536.20823|        null|
|  12001006|  M|             M| 49842.14981|        null|
|  12001007|  M|             M| 123370.4837|           7|
|  12001008|  M|             M| 183331.9827|        null|
|  12001009|  M|             M| 93803.84407|        null|
|  12001010|  M|             M| 35880.96962|           5|
|  12001011|  M|             M| 63576.87454|           0|
|  12001012|  M|             M| 108093.2976|           1|
|  12001013|  M|             M| 331757.8472|           5|
|  12001014|  

## Left Join

In [25]:
df_lms. \
    join(df_Customer, df_lms.CUSTOMERID == df_Customer.CUSTOMERID, 'left'). \
    select(df_lms["CUSTOMERID"], df_Customer["SEX"],df_Customer["MARITAL_STATUS"],df_Customer["GROSS_INCOME"],df_Customer["PRE_JOBYEARS"]).\
    show()

+----------+---+--------------+------------+------------+
|CUSTOMERID|SEX|MARITAL_STATUS|GROSS_INCOME|PRE_JOBYEARS|
+----------+---+--------------+------------+------------+
|  12001000|  M|             M| 198375.2218|           8|
|  12001001|  M|             M| 242703.9829|          10|
|  12001002|  M|             M|         0.0|        null|
|  12001003|  M|             M| 365263.5116|        null|
|  12001004|  M|             M|  187342.199|        null|
|  12001005|  M|             M| 53536.20823|        null|
|  12001006|  M|             M| 49842.14981|        null|
|  12001007|  M|             M| 123370.4837|           7|
|  12001008|  M|             M| 183331.9827|        null|
|  12001009|  M|             M| 93803.84407|        null|
|  12001010|  M|             M| 35880.96962|           5|
|  12001011|  M|             M| 63576.87454|           0|
|  12001012|  M|             M| 108093.2976|           1|
|  12001013|  M|             M| 331757.8472|           5|
|  12001014|  

## Right Join

In [27]:
df_lms. \
    join(df_Customer, df_lms.CUSTOMERID == df_Customer.CUSTOMERID, 'right'). \
    select(df_lms["CUSTOMERID"], df_Customer["SEX"],df_Customer["MARITAL_STATUS"],df_Customer["GROSS_INCOME"],df_Customer["PRE_JOBYEARS"]).\
    show()

+----------+---+--------------+------------+------------+
|CUSTOMERID|SEX|MARITAL_STATUS|GROSS_INCOME|PRE_JOBYEARS|
+----------+---+--------------+------------+------------+
|  12001000|  M|             M| 198375.2218|           8|
|  12001001|  M|             M| 242703.9829|          10|
|  12001002|  M|             M|         0.0|        null|
|  12001003|  M|             M| 365263.5116|        null|
|  12001004|  M|             M|  187342.199|        null|
|  12001005|  M|             M| 53536.20823|        null|
|  12001006|  M|             M| 49842.14981|        null|
|  12001007|  M|             M| 123370.4837|           7|
|  12001008|  M|             M| 183331.9827|        null|
|  12001009|  M|             M| 93803.84407|        null|
|  12001010|  M|             M| 35880.96962|           5|
|  12001011|  M|             M| 63576.87454|           0|
|  12001012|  M|             M| 108093.2976|           1|
|  12001013|  M|             M| 331757.8472|           5|
|  12001014|  

## Full Join

In [28]:
df_lms. \
    join(df_Customer, df_lms.CUSTOMERID == df_Customer.CUSTOMERID, 'full'). \
    select(df_lms["CUSTOMERID"], df_Customer["SEX"],df_Customer["MARITAL_STATUS"],df_Customer["GROSS_INCOME"],df_Customer["PRE_JOBYEARS"]).\
    show()

+----------+----+--------------+------------+------------+
|CUSTOMERID| SEX|MARITAL_STATUS|GROSS_INCOME|PRE_JOBYEARS|
+----------+----+--------------+------------+------------+
|  12001106|null|          null|         0.0|        null|
|  12001259|   M|             M| 332600.3293|        null|
|  12001327|   M|             M|         0.0|        null|
|  12001475|   F|             M| 171778.2654|        null|
|  12001499|   M|             M| 100137.9312|        null|
|  12002029|   M|             M| 47449.95505|        null|
|  12002240|   M|             M| 34112.44317|        null|
|  12002637|null|          null|         0.0|        null|
|  12002655|   M|             S|  63054.4236|           2|
|  12002687|   M|             M|  188007.273|        null|
|  12003035|   F|             M| 62937.81796|        null|
|  12003318|   F|             M| 47891.33602|        null|
|  12003733|null|          null|         0.0|        null|
|  12003749|   M|             M| 98522.53688|        nul

## Performance Tuning

### Brodcast Join

In [34]:
from pyspark.sql.functions import broadcast

In [37]:
peopleDF = (
  ("andrea", "medellin"),
  ("rodolfo", "medellin"),
  ("abdul", "bangalore")
)

peopleDF_a = spark.sparkContext.parallelize(peopleDF)
peopleDF_b = spark.createDataFrame(peopleDF_a, ["first_name", "city"])

peopleDF_b.show()

+----------+---------+
|first_name|     city|
+----------+---------+
|    andrea| medellin|
|   rodolfo| medellin|
|     abdul|bangalore|
+----------+---------+



In [38]:
citiesDF = (
  ("medellin", "colombia", 2.5),
  ("bangalore", "india", 12.3)
)


citiesDF_a = spark.sparkContext.parallelize(citiesDF)
citiesDF_b = spark.createDataFrame(citiesDF_a, ("city", "country", "population"))

citiesDF_b.show()

+---------+--------+----------+
|     city| country|population|
+---------+--------+----------+
| medellin|colombia|       2.5|
|bangalore|   india|      12.3|
+---------+--------+----------+



In [39]:
# Brodcast Hash Join
peopleDF_b_j = peopleDF_b.join(
  broadcast(citiesDF_b),
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#712 = city#722)
:- LogicalRDD [first_name#711, city#712], false
+- ResolvedHint (broadcast)
   +- LogicalRDD [city#722, country#723, population#724], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#712 = city#722)
:- LogicalRDD [first_name#711, city#712], false
+- ResolvedHint (broadcast)
   +- LogicalRDD [city#722, country#723, population#724], false

== Optimized Logical Plan ==
Join Inner, (city#712 = city#722)
:- Filter isnotnull(city#712)
:  +- LogicalRDD [first_name#711, city#712], false
+- ResolvedHint (broadcast)
   +- Filter isnotnull(city#722)
      +- LogicalRDD [city#722, country#723, population#724], false

== Physical Plan ==
*(2) BroadcastHashJoin [city#712], [city#722], Inner, BuildRight
:- *(2) Filter isnotnull(city#712)
:  +- Scan ExistingRDD[first_name#711,city#712]
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, 

In [40]:
## Sort Merge Join
peopleDF_b_j = peopleDF_b.join(
  citiesDF_b,
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#712 = city#722)
:- LogicalRDD [first_name#711, city#712], false
+- LogicalRDD [city#722, country#723, population#724], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#712 = city#722)
:- LogicalRDD [first_name#711, city#712], false
+- LogicalRDD [city#722, country#723, population#724], false

== Optimized Logical Plan ==
Join Inner, (city#712 = city#722)
:- Filter isnotnull(city#712)
:  +- LogicalRDD [first_name#711, city#712], false
+- Filter isnotnull(city#722)
   +- LogicalRDD [city#722, country#723, population#724], false

== Physical Plan ==
*(5) SortMergeJoin [city#712], [city#722], Inner
:- *(2) Sort [city#712 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(city#712, 200)
:     +- *(1) Filter isnotnull(city#712)
:        +- Scan ExistingRDD[first_name#711,city#712]
+- *(4) Sort [city#722 ASC NULLS FIRST], false, 0
   +- Exchange hashparti

## Automatic Detection

- In many cases, Spark can automatically detect whether to use a broadcast join or not, depending on the size of the data. If Spark can detect that one of the joined DataFrames is small (10 MB by default), Spark will automatically broadcast it for us. The code below:

In [44]:
f = df_lms.join(df_Customer,"CUSTOMERID")
f.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(CUSTOMERID))
:- Relation[AGREEMENTID#96,CUSTOMERID#97,LOAN_AMT#98,NET_DISBURSED_AMT#99,INTEREST_START_DATE#100,CURRENT_ROI#101,ORIGNAL_ROI#102,CURRENT_TENOR#103,ORIGNAL_TENOR#104,DUEDAY#105,AUTHORIZATIONDATE#106,CITY#107,PRE_EMI_DUEAMT#108,PRE_EMI_RECEIVED_AMT#109,PRE_EMI_OS_AMOUNT#110,EMI_DUEAMT#111,EMI_RECEIVED_AMT#112,EMI_OS_AMOUNT#113,EXCESS_AVAILABLE#114,EXCESS_ADJUSTED_AMT#115,BALANCE_EXCESS#116,NET_RECEIVABLE#117,OUTSTANDING_PRINCIPAL#118,PAID_PRINCIPAL#119,... 14 more fields] csv
+- Relation[CUSTOMERID#210,CUST_CONSTTYPE_ID#211,CUST_CATEGORYID#212,PROFESSION#213,AGE#214,SEX#215,MARITAL_STATUS#216,QUALIFICATION#217,NO_OF_DEPENDENT#218,OCCUPATION#219,POSITION#220,GROSS_INCOME#221,PRE_JOBYEARS#222,NETTAKEHOMEINCOME#223,BRANCH_PINCODE#224] csv

== Analyzed Logical Plan ==
CUSTOMERID: string, AGREEMENTID: int, LOAN_AMT: string, NET_DISBURSED_AMT: string, INTEREST_START_DATE: string, CURRENT_ROI: double, ORIGNAL_ROI: double, CURR

## Configuring Broadcast Join Detection

- The threshold for automatic broadcast join detection can be tuned or disabled.
- The configuration is spark.sql.autoBroadcastJoinThreshold, and the value is taken in bytes. If you want to configure it to another number, we can set it in the SparkSession:

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)

In [46]:
# Disable
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 

In [ ]:
# This time it will not use Brodcast Hash join
f = df_lms.join(df_Customer,"CUSTOMERID")
f.explain(True)